In [24]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np 
from scipy.stats import norm
from math import log, sqrt, exp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

import pandas as pd



In [26]:
def bs_call_price(S, K, r, sigma, t, T):
    """
    Black–Scholes formula for a European call option.
    S: current underlying price
    K: strike
    r: risk-free rate
    sigma: volatility
    t: current time
    T: maturity time
    """
    tau = T - t  # time to maturity
    if tau <= 0:
        return max(S - K, 0)  # option payoff at/after maturity
    
    d1 = (log(S/K) + (r + 0.5*sigma**2)*tau) / (sigma*sqrt(tau))
    d2 = d1 - sigma*sqrt(tau)
    
    call = S*norm.cdf(d1) - K*exp(-r*tau)*norm.cdf(d2)
    return call

def bs_call_delta(S, K, r, sigma, t, T):
    """
    Delta for a European call option (partial derivative of price wrt S).
    """
    tau = T - t
    if tau <= 0:
        return (S>K)* 1.0
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*tau) / (sigma*np.sqrt(tau))
    return norm.cdf(d1)


In [27]:

# ------------------------------
# Parameters for the simulation
# ------------------------------
S0 = 100.0      # initial underlying price
K  = 100.0      # strike
r  = 0.02       # annual risk-free rate
sigma = 0.20    # volatility (20%)
T  = 1.0        # maturity in years (1 year)
steps = 20     # number of discrete hedging steps
dt = T / steps  # length of each time step
nb_simul = 100000
costs = [0.0,0.001,0.002,0.005]



In [31]:
#Simulated path 
def simulation_path(S0,K,r,sigma,steps,nb_simul):
    times = np.linspace(0, T, steps+1)
    Z = np.random.normal(0,1,(nb_simul,steps+1))
    Z = np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    Z[:,0]=S0
    Z = Z.cumprod(axis = 1)
    return(Z.T)
Z = simulation_path(S0,K,r,sigma,steps,nb_simul)

In [32]:
"""
# ----------------------
# Delta-hedging
# ----------------------
# We SELL 1 call option at t=0, so we get its premium initially.
premium = bs_call_price(S0, K, r, sigma, 0.0, T)"""
final_portfolios_values = []
times= list(np.linspace(0,20,21,dtype ='int'))

for cost in costs : 
    # Our hedge: we hold Delta shares of the underlying (continuously updated).
    # Start: compute initial delta at t=0
    delta_old = bs_call_delta(S0, K, r, sigma, 0.0, T)
    premium = bs_call_price(S0, K, r, sigma, 0.0, T)
    # We'll track the value of our hedging portfolio:
    #   - "option value" is negative (we sold the call).
    #   - "stock holding" is delta shares.
    #   - "cash" starts with the option premium minus the cost to buy delta shares.
    portfolio_value = premium - delta_old*S0
    # To keep track of PnL, note that at each step we:
    #  - Revalue the option
    #  - Rebalance the hedge (update delta)
    #  - Gains/losses go into portfolio_value
    for i in range(1, steps+1):
        t = times[i]
        # New delta
        delta_new = bs_call_delta(Z[:,i], K, r, sigma, t, T)
        
        # Rebalance cost: We go from delta_old shares to delta_new shares.
        # The price of the shares is S[i]. 
        # So the immediate cashflow from rebalancing is: (delta_old - delta_new)*S[i]
        # (If delta_new > delta_old, we are buying shares, so we spend money (negative).)
        rebalance_cashflow = (delta_old - delta_new)*Z[:,i] - cost*np.abs((delta_old - delta_new)*Z[:,i])
        
        # Update portfolio value by that cashflow
        portfolio_value += rebalance_cashflow
        # Move forward in time: accrue risk-free interest over dt
        # (Very rough discrete approximation: portfolio_value *= exp(r*dt))
        # We'll do that only if we want to incorporate interest on any cash part:
        portfolio_value *= exp(r*dt)
        
        # Update old delta
        delta_old = delta_new

    # ---------------------------
    # Final payout of the option
    # ---------------------------
    # At maturity, the call option we sold costs us: payoff = max(S(T) - K, 0).
    final_option_payoff = (Z[:,-1]-K >0)*(Z[:,-1]-K) 

    # The final stock holding is delta_old shares worth delta_old * S[-1].
    # We can liquidate them at maturity. So the final net in the portfolio is:
    final_portfolio_value = portfolio_value + delta_old * Z[:,-1] - final_option_payoff -cost*delta_old * Z[:,-1]
    final_portfolios_values.append(final_portfolio_value)

In [33]:
import plotly.graph_objects as go


In [34]:
fig = go.Figure()
for i in range(len(costs)):

    fig.add_trace(go.Histogram(x=final_portfolios_values[i]))

# Overlay both histograms
fig.update_layout(barmode = 'overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

# Make the plot for delta for different time to maturity and different stike 

# try to construct a small neural network that takes time to matuirity and moneyness and get the delta 

In [40]:
# We areg goin to construct a small neural networks that takes in input time to matuirity and moneyness and get the delta back 
# First we construct our train set 
# We are going to try with 20 periods 
S = simulation_path(S0,K,r,sigma,steps,nb_simul)
ttm = list(np.linspace(0,20,21,dtype ='int'))
l = [[time,s] for (time,s) in zip(ttm,S)]
dataset = np.zeros(((steps+1)*nb_simul,3))

In [42]:
dataset[:,0] = ttm*nb_simul
dataset[:,1] = S.T.reshape(2100000,)
dataset[:,2] = [bs_call_delta(dataset[i,1],K,r,sigma,dataset[i,0],20) for i in range((nb_simul*(steps+1)))]


In [43]:
dataset[:,1] = K/dataset[:,1]

# Buil a neural network to predict y 

In [47]:
X_train = dataset[:200000,:2]
Y_train = dataset[:200000,2]
X_test = dataset[200000:,:2]
Y_test = dataset[200000:,2]

In [48]:
X_train

array([[ 0.        ,  1.        ],
       [ 1.        ,  0.9646165 ],
       [ 2.        ,  0.95229042],
       ...,
       [14.        ,  1.14062537],
       [15.        ,  1.15160179],
       [16.        ,  1.08261165]])

In [49]:

model = keras.Sequential([
    layers.Dense(32,input_shape=(2,)),  
    layers.BatchNormalization(),                         
    layers.Activation('relu'),
    layers.Dropout(0.2),

    layers.Dense(64, use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.2),

    layers.Dense(32, use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.2),

    # Output layer with 1 neuron (sigmoid -> output in [0, 1])
    layers.Dense(1, activation='sigmoid')
])

def custom_loss(y_true, y_pred):
    # Example: Mean Squared Error with an additional penalty
    mse = tf.reduce_mean(tf.square(y_pred- y_true))

     # Add a penalty term
    return mse + 0.01* tf.sqrt((tf.reduce_mean(tf.square(y_pred)) - tf.square(tf.reduce_mean(y_pred))))

# 3. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=custom_loss,
              metrics=['accuracy'])

# 4. Train the model
history = model.fit(
    X_train, Y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,  # 20% of data for validation
    verbose=1
)


/Users/mcbookairdebat/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

2025-01-18 11:33:06.932861: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-01-18 11:33:06.933145: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-01-18 11:33:06.933721: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-01-18 11:33:06.934139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-18 11:33:06.934631: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GP

Epoch 1/5


2025-01-18 11:33:08.573071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 25ms/step - accuracy: 0.0377 - loss: 0.0302 - val_accuracy: 0.0423 - val_loss: 0.0070
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 29ms/step - accuracy: 0.0454 - loss: 0.0079 - val_accuracy: 0.0451 - val_loss: 0.0053
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 30ms/step - accuracy: 0.0452 - loss: 0.0063 - val_accuracy: 0.0415 - val_loss: 0.0078
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.0448 - loss: 0.0057 - val_accuracy: 0.0469 - val_loss: 0.0029
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.0455 - loss: 0.0053 - val_accuracy: 0.0454 - val_loss: 0.0035


NameError: name 'X' is not defined

In [50]:

# 5. Evaluate on the entire dataset
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f"Final Training Loss: {loss:.4f}")
print(f"Final Training Accuracy: {accuracy:.4f}")


KeyboardInterrupt: 

In [51]:
X_test

array([[17.        ,  1.09018821],
       [18.        ,  1.11035535],
       [19.        ,  1.12877259],
       ...,
       [18.        ,  0.83353375],
       [19.        ,  0.80757817],
       [20.        ,  0.8421496 ]])

In [95]:
moneyness = np.linspace(0, 2.5, 1000)
time_to_maturity = np.linspace(0, 20, 21)[::-1]
Moneyness,Time = np.meshgrid(moneyness,time_to_maturity)
data= {
    'Time to Maturity': Time.ravel(),'Moneyness': Moneyness.ravel()
    
}
df = pd.DataFrame(data)
S_test = K/Moneyness
option_type ='call'
def vectorized_delta(option_type, S, K, T, r, sigma):
    """
    Vectorized Delta calculation for arrays of S and T.
    """
    # Avoid division by zero
    T = np.where(T <= 0, 1e-6, T)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    

    delta = norm.cdf(d1)
   
    
    return delta
Delta = vectorized_delta('call', S_test, K, Time, r, sigma)
df['Delta_bs'] = Delta.flatten()

fig = go.Figure(data=[go.Surface(
    x=Moneyness,
    y=Time,
    z=Delta,
    colorscale='Viridis',
    colorbar=dict(title='Delta'),
    contours = {
        "x": {"show": True, "start":0.5, "end":1.5, "size":0.1},
        "y": {"show": True, "start":0.0, "end":2.0, "size":0.2},
        "z": {"show": True, "size":0.05}
    }
)])

# Update layout for better aesthetics
fig.update_layout(
    title=f'Option Delta Surface ({option_type.capitalize()})',
    scene=dict(
        xaxis_title='Moneyness (S/K)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Delta',
        xaxis=dict(nticks=10, range=[0.25, 2.5]),
        yaxis=dict(nticks=10, range=[0, 20]),
        zaxis=dict(nticks=10, range=[0, 1] if option_type == 'call' else [-1, 0]),
    ),
    width=800,
    height=700,
    autosize=False
)

# Add interactive hover information
fig.update_traces(
    hovertemplate='Moneyness: %{x:.2f}<br>Time: %{y:.2f} yr<br>Delta: %{z:.2f}<extra></extra>'
)

# Show the figure
fig.show()

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_14667/2362768988.py:9: RuntimeWarning:

divide by zero encountered in divide



In [96]:
Delta.shape

(21, 1000)

In [70]:
df

,Time to Maturity,Moneyness,Delta_bs
0,20.0,0.000000,1.0
1,20.0,0.002503,1.0
2,20.0,0.005005,1.0
3,20.0,0.007508,1.0
4,20.0,0.010010,1.0
...,...,...,...
20995,0.0,2.489990,0.0
20996,0.0,2.492492,0.0
20997,0.0,2.494995,0.0
20998,0.0,2.497497,0.0


In [71]:
df

,Time to Maturity,Moneyness,Delta_bs
0,20.0,0.000000,1.0
1,20.0,0.002503,1.0
2,20.0,0.005005,1.0
3,20.0,0.007508,1.0
4,20.0,0.010010,1.0
...,...,...,...
20995,0.0,2.489990,0.0
20996,0.0,2.492492,0.0
20997,0.0,2.494995,0.0
20998,0.0,2.497497,0.0


In [98]:
df['Delta'] = model.predict(np.array(df.iloc[:,:2]))

657/657 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [99]:
df['Calibrage_BS'] = df.Delta_bs - df.Delta

In [78]:
df

,Time to Maturity,Moneyness,Delta_bs,Delta,Calibrage_BS
0,20.0,0.000000,1.0,0.999877,0.000123
1,20.0,0.002503,1.0,0.999875,0.000125
2,20.0,0.005005,1.0,0.999873,0.000127
3,20.0,0.007508,1.0,0.999871,0.000129
4,20.0,0.010010,1.0,0.999869,0.000131
...,...,...,...,...,...
20995,0.0,2.489990,0.0,0.307892,-0.307892
20996,0.0,2.492492,0.0,0.307018,-0.307018
20997,0.0,2.494995,0.0,0.306146,-0.306146
20998,0.0,2.497497,0.0,0.305275,-0.305275


In [100]:
import numpy as np
import plotly.graph_objects as go

# Assuming your DataFrame is named df
# Extract the columns
time_to_maturity = np.array(df['Time to Maturity'])
moneyness = np.array(df['Moneyness'])
delta = np.array(df['Calibrage_BS']).reshape(21,1000)


array([20., 20., 20., ...,  0.,  0.,  0.])

In [103]:

# Create an interactive 3D surface plot
fig = go.Figure(data=[go.Surface(
    z=delta,x=Moneyness,
    y=Time,
    colorscale= 'twilight', colorbar=dict(title='Delta')
)])

# Add layout details
fig.update_layout(
    title=f'Option Delta Surface ({option_type.capitalize()})',
    scene=dict(
        xaxis_title='Moneyness (S/K)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Delta_difference',
        xaxis=dict(nticks=10, range=[0.25, 2.5]),
        yaxis=dict(nticks=10, range=[0, 20]),
        zaxis=dict(nticks=10, range=[-1, 1] if option_type == 'call' else [-1, 0]),
    ),
    width=800,
    height=700,
    autosize=False
)


# Show the plot
fig.show()



Try to train a model on a total path and train the model to be not to far that the full delta but also to maximise the reward coming from the delta hedging 

## 1. Dataset 

In [116]:
S = simulation_path(S0,K,r,sigma,steps,nb_simul)/K
ttm = list(np.linspace(0,20,21,dtype ='int'))[::-1]*nb_simul

In [117]:
dataset = pd.DataFrame(S.T.flatten(),columns=['S'])
dataset['ttm'] = ttm

In [124]:
dataset.iloc[1,0]

0.9619779278804714

In [127]:
dataset['delta'] = [bs_call_delta(dataset.iloc[i,0]*K,K,r,sigma,dataset.iloc[i,1],20) for i in range((nb_simul*(steps+1)))]


In [129]:
dataset

,S,ttm,delta
0,1.000000,20,0.000000
1,0.961978,19,0.502466
2,0.908317,18,0.477217
3,0.942422,17,0.569546
4,0.916687,16,0.572416
...,...,...,...
2099995,0.830660,4,0.715010
2099996,0.837826,3,0.729084
2099997,0.891428,2,0.762102
2099998,0.988141,1,0.804580


In [1]:
import numpy as np

def generate_paths(num_paths, num_steps, T, S0, r, sigma, random_seed=None):
    """
    Generates sample paths for a Geometric Brownian Motion using NumPy.
    
    Args:
        num_paths   : int, number of simulated paths
        num_steps   : int, number of time steps
        T           : float, total time horizon (e.g., 1.0 for 1 year)
        S0          : float, initial asset price
        r           : float, risk-free interest rate
        sigma       : float, volatility
        random_seed : int or None, for reproducibility of random draws

    Returns:
        A NumPy array of shape (num_paths, num_steps+1) containing the simulated paths.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    dt = T / num_steps
    # Initialize paths array
    paths = np.zeros((num_paths, num_steps + 1))
    # Set the initial value
    paths[:, 0] = S0

    for step in range(num_steps):
        Z = np.random.normal(0.0, 1.0, size=num_paths)
        # Euler-Maruyama (log form) update
        paths[:, step+1] = paths[:, step] * np.exp(
            (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
        )

    return paths


In [14]:
import math
from scipy.stats import norm

def bs_call_delta(moneyness, T, r, sigma):
    """
    Computes the Black-Scholes delta for a call option given moneyness = K/S.
    
    Args:
        moneyness (float or np.array): K/S
        T        (float or np.array): time to maturity
        r        : float, risk-free rate
        sigma    : float, volatility

    Returns:
        A NumPy array (or float) of the delta values.
    """
    M = np.array(moneyness, ndmin=1)  # ensure array
    time_ = np.array(T, ndmin=1)

    # To avoid dividing by zero if T=0, add a small epsilon
    epsilon = 1e-12
    time_ = np.maximum(time_, epsilon)

    # d1 = [-ln(M) + (r + 0.5*sigma^2)*T] / [sigma * sqrt(T)]
    d1 = (-np.log(M) + (r + 0.5 * sigma**2) * time_) / (sigma * np.sqrt(time_))
    
    # Standard normal CDF using the error function
    # Phi(x) = 0.5 * [1 + erf(x / sqrt(2))]
    def phi(x):
        return norm.cdf(x)
    
    delta = phi(d1)
    print(delta)
    # If inputs were scalars, return scalar
    if np.isscalar(moneyness) and np.isscalar(T):
        return delta[0]
    return delta


In [15]:
import plotly.graph_objects as go

def plot_delta_surface(r, sigma, moneyness_min=0.5, moneyness_max=1.5, num_moneyness=50):
    """
    Plots an interactive 3D surface of Black-Scholes call delta vs moneyness and time to maturity.
    
    Args:
        r              : float, risk-free rate
        sigma          : float, volatility
        moneyness_min  : float, lower bound of the moneyness range
        moneyness_max  : float, upper bound of the moneyness range
        num_moneyness  : int, number of points in the moneyness grid
    """
    # Generate ranges
    moneyness_values = np.linspace(moneyness_min, moneyness_max, num_moneyness)
    # 20 steps between 0 and 1 for time to maturity
    time_values = np.linspace(1e-3, 1.0, 20)

    # Create a meshgrid
    M_grid, T_grid = np.meshgrid(moneyness_values, time_values, indexing='xy')

    # Compute delta on the grid
    Delta_grid = bs_call_delta(M_grid, T_grid, r, sigma)

    # Create a surface plot
    fig = go.Figure(data=[go.Surface(
        x=M_grid,
        y=T_grid,
        z=Delta_grid,
        colorscale='Viridis'
    )])
    
    fig.update_layout(
        title='Black–Scholes Delta Surface',
        scene=dict(
            xaxis_title='Moneyness (K/S)',
            yaxis_title='Time to Maturity',
            zaxis_title='Delta'
        ),
        autosize=True
    )
    
    fig.show()


In [16]:

paths = generate_paths(
    num_paths=5,
    num_steps=5,
    T=1.0,
    S0=100.0,
    r=0.05,
    sigma=0.2,
    random_seed=42
)
print("Sample GBM Paths:\n", paths)

# 2) Compute delta for a single moneyness/time
single_delta = bs_call_delta(moneyness=0.95, T=0.5, r=0.05, sigma=0.2)
print("\nDelta for moneyness=0.95, T=0.5:", single_delta)

# 3) Plot the 3D surface of the delta
plot_delta_surface(r=0.05, sigma=0.2, moneyness_min=0.8, moneyness_max=1.2)


Sample GBM Paths:
 [[100.         105.17205735 103.61227469 100.00362805  95.67087736
  109.728107  ]
 [100.          99.36534897 115.12891945 111.09619012 102.08502396
  100.64625951]
 [100.         106.60188665 114.86329233 118.08260505 122.17953462
  123.65945594]
 [100.         115.28347758 111.20808122  94.28047012  87.44918853
   77.44938067]
 [100.          98.51677827 104.03786475  89.70025177  79.53150979
   76.20767821]]
[0.72913061]

Delta for moneyness=0.95, T=0.5: 0.7291306113288041
[[1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  9.99999998e-001 9.99997830e-001 9.99464660e-001 9.74949906e-001
  7.44651063e-001 2.63372920e-001 2.78572168e-002 7.28132588e-004
  4.43279714e-006 6.23922059e-009 2.0